1) Una importante empresa de reventa de dispositivos electrónicos se
encuentra realizando preparativos para el próximo Black Friday en su
canal de ecommerce. Como parte de su equipo de análisis de datos tu
misión es calcular la siguientes métricas usando pyspark:

A) Tiempo de sesión promedio de usuarios en el sitio. (siendo
el tiempo de una sesión de un usuario el tiempo acumulado
de todos sus eventos para cada sesión) (7 puntos) #asumo que quiere el tiempo promedio de sesion de cada usuario

B) Bounce Rate (cantidad de sesiones que solamente
realizaron un evento % cantidad de sesiones totales) (13
puntos)

Esto debe realizarse a partir de la información recolectada por su
plataforma de analytics la cual es accesible en un RDD del siguiente
formato (id_de_cliente, id_de_sesion, url,
tipo_de_evento, metadata,
tiempo_transcurrido_desde_anterior_evento).
(****) ( 20pts)

In [50]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

salesRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/eventos.csv', header=True).rdd

salesRdd = salesRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), pd.to_numeric(x[1], 'coerce')
                                   , x[2], x[3], x[4], pd.to_numeric(x[5], 'coerce'))).filter(lambda x: x[3] is not None).cache()

sessionsAndTime = salesRdd.map(lambda x: ((x[0], x[1]), x[5])).reduceByKey(lambda x,y: x + y) \
        .map(lambda x: (x[1], 1)).reduce(lambda x,y: (x[0] + y[0], x[1] + y[1]))

sessionsAndTime[0] / sessionsAndTime[1]

353.5013623978202

In [51]:
sessionsWithOneEvent = salesRdd.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(lambda x,y: x + y) \
        .filter(lambda x: x[1] == 1).count()

sessionsWithOneEvent / sessionsAndTime[1]

0.7956403269754768